# Warsztat 3: End-to-End Bronze → Silver → Gold Pipeline

**Cel warsztatu:**
- Implementacja kompletnego pipeline'u Bronze → Silver → Gold
- Integracja wielu źródeł danych (customers, orders, products)
- Transformacje biznesowe i agregacje w architekturze medalionowej
- Optymalizacja i monitorowanie pipeline'u

**Czas:** 120 minut

**Architektura docelowa:**
```
Bronze (Raw Data) Silver (Cleansed) Gold (Analytics)
├── customers.csv → ├── customers_clean → ├── customer_analytics
├── orders.json → ├── orders_clean → ├── product_performance 
└── products.parquet→ └── products_clean → └── sales_summary
```

---

## 📚 Inicjalizacja środowiska

In [ ]:
%run ../../00_setup

**Wyjaśnienie inicjalizacji:**
- Skrypt `00_setup` konfiguruje per-user izolację (katalogi, schematy)
- Automatycznie tworzy zmienne środowiskowe: `CATALOG`, `BRONZE_SCHEMA`, `SILVER_SCHEMA`, `GOLD_SCHEMA`
- Zapewnia, że każdy użytkownik pracuje w izolowanym namespace

## Część 1: Warstwa Bronze - Raw Data Ingestion

### Zadanie 1.1: Ingestion danych klientów (CSV)

**Instrukcje:**
1. Wczytaj dane z `customers.csv` używając Auto Loader
2. Dodaj kolumny metadata: `_source_file`, `_ingestion_timestamp`
3. Zapisz do `bronze_customers_pipeline`

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Ścieżki do plików źródłowych w dataset
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

from pyspark.sql.functions import current_timestamp, input_file_name

# TODO: Auto Loader dla customers
bronze_customers_stream = (
 spark.readStream
 .format("____") # cloudFiles
 .option("cloudFiles.format", "____") # csv
 .option("cloudFiles.schemaLocation", f"{CHECKPOINT_PATH}/customers_schema")
 .option("header", "true")
 .load(f"{SOURCE_DATA_PATH}/____") # customers.csv
 .withColumn("_source_file", ____) # input_file_name()
 .withColumn("_ingestion_timestamp", ____) # current_timestamp()
)

**Konfiguracja środowiska:**
- Importujemy funkcje PySpark do transformacji danych
- Definiujemy ścieżki do plików źródłowych w dataset
- Pliki są w różnych formatach: CSV (customers), JSON (orders), Parquet (products)
- Użyjemy rzeczywistych kolumn z dataset'u

In [ ]:
# TODO: Krok 1 - Wczytaj customers z CSV
customers_raw = (
 spark.read
 .format("____") # csv
 .option("header", "____") # true
 .option("inferSchema", "____") # true
 .load(____) # CUSTOMERS_CSV
)

# TODO: Zapisz do Bronze
query_bronze_customers = (
 bronze_customers_stream.writeStream
 .format("____") # delta
 .outputMode("____") # append
 .option("checkpointLocation", f"{____}/bronze_customers") # CHECKPOINT_PATH
 .table(f"{CATALOG}.{SCHEMA}.bronze_customers_pipeline")
)

**Krok 3: Zapis do Bronze**
- `.format("delta")` - zapisujemy w formacie Delta Lake
- `.mode("overwrite")` - zastępujemy istniejące dane
- `overwriteSchema="true"` - pozwalamy na zmiany w schemacie
- Tabela zostanie utworzona w schemacie Bronze z prefiksem użytkownika

In [ ]:
# TODO: Krok 3 - Zapisz customers do Bronze
(
 customers_bronze
 .write
 .format("____") # delta
 .mode("____") # overwrite
 .option("overwriteSchema", "true")
 .saveAsTable(f"{____}.customers_bronze") # BRONZE_SCHEMA
)

**Krok 2: Audit metadata dla Bronze**
- `_bronze_ingest_timestamp` - użyj `current_timestamp()` - kiedy dane zostały załadowane
- `_bronze_source_file` - użyj `input_file_name()` - z jakiego pliku pochodzą dane
- `_bronze_ingested_by` - użyj `raw_user` - kto załadował dane
- Te kolumny pomagają w śledzeniu pochodzenia danych (data lineage)

In [ ]:
# TODO: Krok 2 - Dodaj audit metadata dla customers
customers_bronze = (
 customers_raw
 .withColumn("_bronze_ingest_timestamp", F.____) # current_timestamp()
 .withColumn("_bronze_source_file", F.____) # input_file_name()
 .withColumn("_bronze_ingested_by", F.lit(____)) # raw_user
)

**Krok 1: Wczytanie customers (CSV)**
- Używamy `.format("csv")` dla plików CSV
- `header="true"` - pierwszy wiersz zawiera nazwy kolumn
- `inferSchema="true"` - automatyczne wykrywanie typów danych
- Kolumny w pliku: `customer_id`, `first_name`, `last_name`, `email`, `phone`, `city`, `state`, `country`, `registration_date`, `customer_segment`

### Zadanie 1.2: Ingestion zamówień (JSON)

**Instrukcje:**
1. Wczytaj dane z `orders_batch.json` używając Auto Loader
2. Dodaj schema hints dla `order_date` (DATE) i `total_amount` (DOUBLE)
3. Zapisz do `bronze_orders_pipeline`

In [ ]:
# TODO: Auto Loader dla orders
bronze_orders_stream = (
 spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "____") # json
 .option("cloudFiles.schemaLocation", f"{CHECKPOINT_PATH}/orders_schema")
 .option("cloudFiles.schemaHints", "____") # "order_date DATE, total_amount DOUBLE"
 .load(f"{SOURCE_DATA_PATH}/____") # orders_batch.json
 .withColumn("_source_file", input_file_name())
 .withColumn("_ingestion_timestamp", current_timestamp())
)

# TODO: Krok 4 - Wczytaj orders z JSON
orders_raw = (
 spark.read
 .format("____") # json
 .option("multiLine", "____") # false - każda linia to osobny JSON
 .load(____) # ORDERS_JSON
)

**Krok 4: Wczytanie orders (JSON)**
- Używamy `.format("json")` dla plików JSON
- `multiLine="false"` - każda linia zawiera osobny obiekt JSON
- Kolumny w orders: `order_id`, `customer_id`, `product_id`, `store_id`, `order_datetime`, `quantity`, `unit_price`, `discount_percent`, `total_amount`, `payment_method`

In [ ]:
# TODO: Zapisz do Bronze
query_bronze_orders = (
 bronze_orders_stream.writeStream
 .format("delta")
 .outputMode("append")
 .option("checkpointLocation", f"{CHECKPOINT_PATH}/____") # bronze_orders
 .table(f"{CATALOG}.{SCHEMA}.bronze_orders_pipeline")
)

# TODO: Krok 5 - Dodaj audit metadata dla orders
orders_bronze = (
 orders_raw
 .withColumn("_bronze_ingest_timestamp", F.current_timestamp())
 .withColumn("_bronze_source_file", F.input_file_name())
 .withColumn("_bronze_ingested_by", F.lit(____) # raw_user
)

**Krok 6: Zapis orders do Bronze**
- Identyczna procedura jak dla customers
- Uzupełnij `"delta"` i `"overwrite"` w miejscach z `____`
- Tabela `orders_bronze` zostanie utworzona w schemacie Bronze

In [ ]:
# TODO: Krok 6 - Zapisz orders do Bronze
(
 orders_bronze
 .write
 .format("____") # delta
 .mode("____") # overwrite
 .option("overwriteSchema", "true")
 .saveAsTable(f"{BRONZE_SCHEMA}.orders_bronze")
)

**Krok 5: Audit metadata dla orders**
- Dodajemy te same kolumny audit co dla customers
- Użyj `raw_user` w miejscu `____` 
- To zapewnia spójne śledzenie pochodzenia danych we wszystkich tabelach Bronze

### Zadanie 1.3: Ingestion produktów (Parquet)

**Instrukcje:**
1. Wczytaj dane z `products.parquet` używając COPY INTO (batch)
2. Dodaj kolumny metadata
3. Zapisz do `bronze_products_pipeline`

In [ ]:
# TODO: Utwórz tabelę Bronze dla produktów
spark.sql(f"""
 CREATE TABLE IF NOT EXISTS {CATALOG}.{SCHEMA}.bronze_products_pipeline (
 product_id INT,
 product_name STRING,
 category STRING,
 price DOUBLE,
 stock_quantity INT,
 _source_file STRING,
 _ingestion_timestamp TIMESTAMP
 )
 USING DELTA
 LOCATION '{____}/products_pipeline' -- BRONZE_PATH
""")

# TODO: Krok 7 - Wczytaj products z Parquet
products_raw = (
 spark.read
 .format("____") # parquet
 .load(____) # PRODUCTS_PARQUET
)

**Krok 7: Wczytanie products (Parquet)**
- Używamy `.format("parquet")` dla plików Parquet
- Parquet automatycznie zawiera schema, więc nie trzeba jej definiować
- Jest to najbardziej efektywny format dla dużych zbiorów danych
- Kolumny w products: `product_id`, `product_name`, `category`, `price`, `stock_quantity`

In [ ]:
# TODO: COPY INTO dla produktów
spark.sql(f"""
 ____ INTO {CATALOG}.{SCHEMA}.bronze_products_pipeline
 FROM (
 SELECT 
 product_id,
 product_name,
 category,
 price,
 stock_quantity,
 _metadata.file_path as _source_file,
 current_timestamp() as _ingestion_timestamp
 FROM '{____}/____' -- SOURCE_DATA_PATH/products.parquet
 )
 FILEFORMAT = ____ -- PARQUET
""")

# TODO: Krok 8 - Dodaj audit metadata dla products
products_bronze = (
 products_raw
 .withColumn("_bronze_ingest_timestamp", F.current_timestamp()) # current_timestamp()
 .withColumn("_bronze_source_file", F.input_file_name()) # input_file_name()
 .withColumn("_bronze_ingested_by", F.lit(raw_user)) # raw_user
)

**Krok 8: Audit metadata dla products**
- Uzupełnij brakujące funkcje: `current_timestamp()`, `input_file_name()`, `raw_user`
- Spójne kolumny audit we wszystkich tabelach Bronze ułatwiają zarządzanie

In [ ]:
# Weryfikacja warstwy Bronze
import time
time.sleep(15) # Poczekaj na przetworzenie streamów

print("=== Bronze Layer Summary ===")
print(f"Customers: {spark.table(f'{CATALOG}.{SCHEMA}.bronze_customers_pipeline').count()}")
print(f"Orders: {spark.table(f'{CATALOG}.{SCHEMA}.bronze_orders_pipeline').count()}")
print(f"Products: {spark.table(f'{CATALOG}.{SCHEMA}.bronze_products_pipeline').count()}")

# TODO: Krok 9 - Zapisz products do Bronze
(
 products_bronze
 .write
 .format("delta") # delta
 .mode("overwrite") # overwrite
 .option("overwriteSchema", "true")
 .saveAsTable(f"{BRONZE_SCHEMA}.products_bronze") # BRONZE_SCHEMA
)

**Krok 10: Weryfikacja warstwy Bronze**
- Sprawdzamy czy wszystkie dane zostały poprawnie załadowane
- Uzupełnij nazwy tabel: `customers_bronze`, `orders_bronze`, `products_bronze`
- Liczby rekordów powinny odpowiadać źródłowym plikom
- To checkpoint przed przejściem do warstwy Silver

In [ ]:
# TODO: Krok 10 - Weryfikacja warstwy Bronze

# Sprawdź liczby rekordów w każdej tabeli
customers_count = spark.table(f"{BRONZE_SCHEMA}.____").count() # customers_bronze
orders_count = spark.table(f"{BRONZE_SCHEMA}.____").count() # orders_bronze 
products_count = spark.table(f"{BRONZE_SCHEMA}.____").count() # products_bronze

# Wyświetl podsumowanie
display(spark.createDataFrame([
 ("Customers Bronze", customers_count),
 ("Orders Bronze", orders_count),
 ("Products Bronze", products_count)
], ["Tabela", "Liczba_rekordów"]))

**Krok 9: Zapis products do Bronze**
- Ostatni krok w warstwie Bronze
- Uzupełnij `"delta"`, `"overwrite"` i `BRONZE_SCHEMA`
- Po tym kroku wszystkie surowe dane będą w warstwie Bronze

---

## Część 2: Warstwa Silver - Data Cleansing & Standardization

### Zadanie 2.1: Silver Customers - Czyszczenie danych

**Instrukcje:**
1. Wczytaj dane z Bronze jako streaming DataFrame
2. Wyczyść dane:
 - Usuń duplikaty po `customer_id`
 - Filtruj nieprawidłowe emaile (muszą zawierać `@`)
 - Standaryzuj nazwy krajów (UPPER)
 - Usuń białe znaki z imion (trim)
3. Dodaj kolumnę `processed_at`
4. Zapisz do `silver_customers_pipeline`

In [ ]:
from pyspark.sql.functions import col, upper, trim, current_timestamp

# TODO: Streaming read z Bronze
bronze_customers = (
 spark.readStream
 .format("____") # delta
 .table(f"{CATALOG}.{SCHEMA}.____") # bronze_customers_pipeline
)

In [ ]:
# TODO: Transformacje Silver
silver_customers = (
 bronze_customers
 .dropDuplicates(["____"]) # customer_id
 .filter(col("____").contains("____")) # email zawiera @
 .filter(col("email").isNotNull()) # email nie jest NULL
 .withColumn("name", ____(col("name"))) # trim
 .withColumn("country", ____(col("country"))) # upper
 .withColumn("processed_at", ____) # current_timestamp
 .select(
 "customer_id", "name", "email", "city", "country", "processed_at"
 )
)

In [ ]:
# TODO: Zapisz do Silver
query_silver_customers = (
 silver_customers.writeStream
 .format("delta")
 .outputMode("____") # append
 .option("checkpointLocation", f"{CHECKPOINT_PATH}/silver_customers")
 .option("mergeSchema", "true")
 .table(f"{CATALOG}.{SCHEMA}.silver_customers_pipeline")
)

### Zadanie 2.2: Silver Orders - Wzbogacenie danych

**Instrukcje:**
1. Wczytaj dane z Bronze
2. Transformacje:
 - Konwertuj `order_date` na DATE
 - Dodaj kolumnę `order_year` (rok z daty)
 - Dodaj kolumnę `order_month` (miesiąc z daty)
 - Kategoryzuj zamówienia: `order_value_category` (LOW < 100, MEDIUM 100-500, HIGH > 500)
3. Zapisz do `silver_orders_pipeline`

In [ ]:
from pyspark.sql.functions import year, month, when, to_date

# TODO: Streaming read z Bronze
bronze_orders = (
 spark.readStream
 .format("delta")
 .table(f"{CATALOG}.{SCHEMA}.____") # bronze_orders_pipeline
)

In [ ]:
# TODO: Transformacje Silver
silver_orders = (
 bronze_orders
 .withColumn("order_date", to_date(col("order_date")))
 .withColumn("order_year", ____(col("order_date"))) # year
 .withColumn("order_month", ____(col("order_date"))) # month
 .withColumn(
 "order_value_category",
 when(col("total_amount") < 100, "____") # LOW
 .when((col("total_amount") >= 100) & (col("total_amount") <= 500), "____") # MEDIUM
 .otherwise("____") # HIGH
 )
 .withColumn("processed_at", current_timestamp())
 .select(
 "order_id", "customer_id", "order_date", "order_year", "order_month",
 "total_amount", "order_value_category", "status", "processed_at"
 )
)

In [ ]:
# TODO: Zapisz do Silver
query_silver_orders = (
 silver_orders.writeStream
 .format("____") # delta
 .outputMode("append")
 .option("checkpointLocation", f"{CHECKPOINT_PATH}/____") # silver_orders
 .table(f"{CATALOG}.{SCHEMA}.silver_orders_pipeline")
)

### Zadanie 2.3: Silver Products - Normalizacja

**Instrukcje:**
1. Wczytaj dane z Bronze (batch)
2. Transformacje:
 - Standaryzuj nazwy kategorii (UPPER)
 - Dodaj kolumnę `is_in_stock` (TRUE jeśli stock_quantity > 0)
 - Dodaj kolumnę `price_tier` (BUDGET < 50, STANDARD 50-200, PREMIUM > 200)
3. Zapisz do `silver_products_pipeline`

In [ ]:
# TODO: Batch read z Bronze
bronze_products = spark.table(f"{CATALOG}.{SCHEMA}.____") # bronze_products_pipeline

In [ ]:
# TODO: Transformacje Silver
silver_products = (
 bronze_products
 .withColumn("category", ____(col("category"))) # upper
 .withColumn("is_in_stock", col("stock_quantity") ____ 0) # >
 .withColumn(
 "price_tier",
 when(col("price") < 50, "____") # BUDGET
 .when((col("price") >= 50) & (col("price") <= 200), "____") # STANDARD
 .otherwise("____") # PREMIUM
 )
 .withColumn("processed_at", current_timestamp())
 .select(
 "product_id", "product_name", "category", "price", "price_tier",
 "stock_quantity", "is_in_stock", "processed_at"
 )
)

In [ ]:
# TODO: Zapisz do Silver (batch)
(
 silver_products.write
 .format("____") # delta
 .mode("____") # overwrite
 .option("path", f"{SILVER_PATH}/products_pipeline")
 .saveAsTable(f"{CATALOG}.{SCHEMA}.silver_products_pipeline")
)

In [ ]:
# Weryfikacja warstwy Silver
time.sleep(15)

print("=== Silver Layer Summary ===")
print(f"Customers: {spark.table(f'{CATALOG}.{SCHEMA}.silver_customers_pipeline').count()}")
print(f"Orders: {spark.table(f'{CATALOG}.{SCHEMA}.silver_orders_pipeline').count()}")
print(f"Products: {spark.table(f'{CATALOG}.{SCHEMA}.silver_products_pipeline').count()}")

---

## 📊 Część 3: Warstwa Gold - Business Analytics

### Zadanie 3.1: Customer Analytics

**Instrukcje:**
1. Stwórz agregację:
 - JOIN silver_customers z silver_orders
 - Grupuj po customer_id, name, country
 - Oblicz: total_orders, total_spent, avg_order_value
2. Zapisz jako `gold_customer_analytics`

In [ ]:
from pyspark.sql.functions import count, sum, avg, round

# TODO: Wczytaj dane Silver
customers = spark.table(f"{CATALOG}.{SCHEMA}.____") # silver_customers_pipeline
orders = spark.table(f"{CATALOG}.{SCHEMA}.____") # silver_orders_pipeline

In [ ]:
# TODO: JOIN i agregacja
customer_analytics = (
 customers.alias("c")
 .join(
 orders.alias("o"),
 col("c.customer_id") == col("o.____"), # customer_id
 "____" # left (aby uwzględnić klientów bez zamówień)
 )
 .groupBy("c.customer_id", "c.name", "c.country")
 .agg(
 ____("o.order_id").alias("total_orders"), # count
 round(____("o.total_amount"), 2).alias("total_spent"), # sum
 round(____("o.total_amount"), 2).alias("avg_order_value") # avg
 )
 .withColumn("processed_at", current_timestamp())
)

display(customer_analytics)

In [ ]:
# TODO: Zapisz do Gold
(
 customer_analytics.write
 .format("delta")
 .mode("____") # overwrite
 .option("path", f"{____}/customer_analytics") # GOLD_PATH
 .saveAsTable(f"{CATALOG}.{SCHEMA}.gold_customer_analytics")
)

### Zadanie 3.2: Product Performance

**Instrukcje:**
1. Stwórz agregację:
 - Kategoria produktu
 - Liczba produktów w kategorii
 - Średnia cena w kategorii
 - Liczba produktów dostępnych (in_stock)
2. Zapisz jako `gold_product_performance`

In [ ]:
# TODO: Wczytaj produkty Silver
products = spark.table(f"{CATALOG}.{SCHEMA}.silver_products_pipeline")

In [ ]:
# TODO: Agregacja po kategorii
product_performance = (
 products
 .groupBy("____") # category
 .agg(
 count("product_id").alias("____"), # total_products
 round(avg("____"), 2).alias("avg_price"), # price
 sum(when(col("is_in_stock") == True, 1).otherwise(0)).alias("____") # products_in_stock
 )
 .withColumn("processed_at", current_timestamp())
 .orderBy(col("total_products").desc())
)

display(product_performance)

In [ ]:
# TODO: Zapisz do Gold
(
 product_performance.write
 .format("____")
 .mode("overwrite")
 .option("path", f"{GOLD_PATH}/____") # product_performance
 .saveAsTable(f"{CATALOG}.{SCHEMA}.gold_product_performance")
)

### Zadanie 3.3: Sales Summary

**Instrukcje:**
1. Stwórz agregację zamówień:
 - Grupuj po roku, miesiącu, kategorii wartości (order_value_category)
 - Oblicz: liczba zamówień, suma przychodów, średnia wartość zamówienia
2. Zapisz jako `gold_sales_summary`

In [ ]:
# TODO: Wczytaj zamówienia Silver
orders = spark.table(f"{CATALOG}.{SCHEMA}.silver_orders_pipeline")

In [ ]:
# TODO: Agregacja sprzedaży
sales_summary = (
 orders
 .groupBy("____", "____", "____") # order_year, order_month, order_value_category
 .agg(
 count("order_id").alias("____"), # total_orders
 round(sum("total_amount"), 2).alias("____"), # total_revenue
 round(avg("total_amount"), 2).alias("____") # avg_order_value
 )
 .withColumn("processed_at", current_timestamp())
 .orderBy("order_year", "order_month", "order_value_category")
)

display(sales_summary)

In [ ]:
# TODO: Zapisz do Gold
(
 sales_summary.write
 .format("delta")
 .mode("____") # overwrite
 .option("path", f"{GOLD_PATH}/sales_summary")
 .saveAsTable(f"{CATALOG}.{SCHEMA}.____") # gold_sales_summary
)

In [ ]:
# Weryfikacja warstwy Gold
print("=== Gold Layer Summary ===")
print(f"Customer Analytics: {spark.table(f'{CATALOG}.{SCHEMA}.gold_customer_analytics').count()}")
print(f"Product Performance: {spark.table(f'{CATALOG}.{SCHEMA}.gold_product_performance').count()}")
print(f"Sales Summary: {spark.table(f'{CATALOG}.{SCHEMA}.gold_sales_summary').count()}")

---

## ⚡ Część 4: Optymalizacja Pipeline'u

### Zadanie 4.1: OPTIMIZE wszystkich tabel

**Instrukcje:**
1. Wykonaj OPTIMIZE dla wszystkich tabel Gold
2. Zastosuj ZORDER dla kluczowych kolumn filtrowania

In [ ]:
# TODO: OPTIMIZE Customer Analytics (ZORDER by country)
spark.sql(f"""
 ____ {CATALOG}.{SCHEMA}.gold_customer_analytics
 ZORDER BY (____)
""")

In [ ]:
# TODO: OPTIMIZE Product Performance (ZORDER by category)
spark.sql(f"""
 OPTIMIZE {CATALOG}.{SCHEMA}.gold_product_performance
 ____ BY (category)
""")

In [ ]:
# TODO: OPTIMIZE Sales Summary (ZORDER by order_year, order_month)
spark.sql(f"""
 OPTIMIZE {CATALOG}.{SCHEMA}.____
 ZORDER BY (order_year, order_month)
""")

### Zadanie 4.2: Data Quality Checks

**Instrukcje:**
1. Sprawdź jakość danych w każdej warstwie:
 - Liczba duplikatów
 - Liczba NULL w kluczowych kolumnach
 - Integralność referencyjna (wszystkie customer_id w orders istnieją w customers)

In [ ]:
# TODO: Sprawdź duplikaty w Silver Customers
duplicates = (
 spark.table(f"{CATALOG}.{SCHEMA}.silver_customers_pipeline")
 .groupBy("____") # customer_id
 .count()
 .filter(col("count") > 1)
)

print(f"Duplikaty w Silver Customers: {duplicates.count()}")

In [ ]:
# TODO: Sprawdź NULL w kluczowych kolumnach
null_checks = spark.sql(f"""
 SELECT 
 'customers' as table_name,
 SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) as null_customer_id,
 SUM(CASE WHEN ____ IS NULL THEN 1 ELSE 0 END) as null_email -- email
 FROM {CATALOG}.{SCHEMA}.silver_customers_pipeline
 
 UNION ALL
 
 SELECT 
 'orders' as table_name,
 SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) as null_order_id,
 SUM(CASE WHEN ____ IS NULL THEN 1 ELSE 0 END) as null_total_amount
 FROM {CATALOG}.{SCHEMA}.silver_orders_pipeline
""")

display(null_checks)

In [ ]:
# TODO: Sprawdź integralność referentialną
orphan_orders = spark.sql(f"""
 SELECT COUNT(*) as orphan_count
 FROM {CATALOG}.{SCHEMA}.silver_orders_pipeline o
 LEFT ANTI JOIN {CATALOG}.{SCHEMA}.silver_customers_pipeline c
 ON o.customer_id = c.____
""")

display(orphan_orders)

### Zadanie 4.3: Monitoring streaming queries

**Instrukcje:**
1. Wyświetl aktywne streaming queries
2. Sprawdź metryki każdego streamu
3. Zatrzymaj wszystkie streamy

In [ ]:
# TODO: Monitoring streamów
active_streams = spark.streams.____ # active

print(f"Liczba aktywnych streamów: {len(active_streams)}")

for stream in active_streams:
 print(f"\n=== Stream: {stream.name} ===")
 print(f"ID: {stream.id}")
 print(f"Status: {stream.status['message']}")
 
 last_progress = stream.lastProgress
 if last_progress:
 print(f"Batch ID: {last_progress['batchId']}")
 print(f"Przetworzone rekordy: {last_progress.get('numInputRows', 0)}")

In [ ]:
# TODO: Zatrzymaj wszystkie streamy
for stream in spark.streams.active:
 print(f"Zatrzymuję stream: {stream.name}")
 stream.____() # stop

print("\nWszystkie streamy zatrzymane!")

---

## 📈 Część 5: Analiza Biznesowa (Bonus)

### Zadanie 5.1: Top 10 Customers by Spend

**Instrukcje:**
1. Wyświetl top 10 klientów według całkowitych wydatków
2. Dodaj informację o kraju i liczbie zamówień

In [ ]:
# TODO: Top 10 klientów
top_customers = spark.sql(f"""
 SELECT 
 name,
 country,
 total_orders,
 total_spent,
 avg_order_value
 FROM {CATALOG}.{SCHEMA}.gold_customer_analytics
 ORDER BY ____ DESC -- total_spent
 LIMIT ____ -- 10
""")

display(top_customers)

### Zadanie 5.2: Category Performance Analysis

**Instrukcje:**
1. Wyświetl performance każdej kategorii produktów
2. Oblicz % produktów dostępnych w magazynie

In [ ]:
# TODO: Analiza kategorii
category_analysis = spark.sql(f"""
 SELECT 
 category,
 total_products,
 avg_price,
 products_in_stock,
 ROUND((____ * 100.0 / ____), 2) as stock_percentage -- products_in_stock / total_products
 FROM {CATALOG}.{SCHEMA}.gold_product_performance
 ORDER BY total_products ____ -- DESC
""")

display(category_analysis)

---

## Podsumowanie warsztatu

**Zrealizowane cele:**
- Kompletny pipeline Bronze → Silver → Gold
- Integracja wielu źródeł danych (CSV, JSON, Parquet)
- Streaming i batch processing
- Transformacje biznesowe i agregacje
- Optymalizacja tabel (OPTIMIZE, ZORDER)
- Data quality checks
- Monitoring pipeline'u

**Architektura finalna:**

```
Bronze (Raw Data)
├── customers (CSV, Auto Loader)
├── orders (JSON, Auto Loader)
└── products (Parquet, COPY INTO)
 ↓
Silver (Cleansed & Enriched)
├── customers (deduplicated, validated)
├── orders (categorized, dated)
└── products (normalized, categorized)
 ↓
Gold (Business Analytics)
├── customer_analytics
├── product_performance
└── sales_summary
```

**Best Practices zastosowane:**
1. Schema evolution z Auto Loader
2. Metadata tracking (_source_file, _ingestion_timestamp)
3. Data quality validation
4. Incremental processing
5. Idempotentność operacji
6. Optymalizacja dla wydajności

---

## 🧹 Cleanup (opcjonalnie)

In [ ]:
# Usuń wszystkie tabele pipeline'u (opcjonalnie)
# tables_to_drop = [
# "bronze_customers_pipeline", "bronze_orders_pipeline", "bronze_products_pipeline",
# "silver_customers_pipeline", "silver_orders_pipeline", "silver_products_pipeline",
# "gold_customer_analytics", "gold_product_performance", "gold_sales_summary"
# ]

# for table in tables_to_drop:
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.{table}")
# print(f"Dropped table: {table}")

In [ ]:
```xml
<VSCode.Cell language="markdown">
# End-to-End Bronze-Silver-Gold Pipeline - Workshop

**Cel szkoleniowy:** Zbudowanie kompletnego, produkcyjnego pipeline'u danych od surowych plików przez Bronze/Silver do Gold layer z optymalizacją i monitoringiem.

**Zakres tematyczny:**
- Raw → Bronze: Ingest z audit metadata
- Bronze → Silver: Cleaning, validation, JSON flattening, deduplikacja
- Silver → Gold: Business aggregates, KPI modeling, denormalizacja
- Performance optimization: partitioning, OPTIMIZE, ZORDER
- Monitoring: data quality metrics, lineage tracking

**Czas trwania:** 120 minut
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Workshop
- **Wymagania techniczne**:
 - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
 - Unity Catalog włączony
 - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
 - Klaster: Standard z minimum 2 workers

**Business Scenario:**
Firma e-commerce potrzebuje pipeline do analizy zamówień:
- Raw data: Orders (JSON), Customers (CSV), Products (Parquet)
- Bronze: Landing zone z audit trail
- Silver: Oczyszczone dane z joinami
- Gold: Daily sales KPIs, customer segments, product performance
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:
</VSCode.Cell>
<VSCode.Cell language="python">
%run ../../00_setup
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:
</VSCode.Cell>
<VSCode.Cell language="python">
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog jako domyślny
spark.sql(f"USE CATALOG {CATALOG}")

# Ścieżki do danych źródłowych
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

print(f"\n=== Ścieżki do danych ===")
print(f"Orders: {ORDERS_JSON}")
print(f"Customers: {CUSTOMERS_CSV}")
print(f"Products: {PRODUCTS_PARQUET}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 1: Bronze Layer - Raw Data Ingestion (20 minut)

**Cel:** Załadowanie surowych danych z różnych formatów do Bronze layer z pełnym audit trail.

### Zadanie 1.1: Bronze - Orders (JSON)

**Instrukcje:**
1. Wczytaj dane z `orders_batch.json` (format: multiline JSON)
2. Dodaj audit columns:
 - `bronze_ingest_timestamp` (current_timestamp)
 - `bronze_source_file` (input_file_name)
 - `bronze_ingested_by` (raw_user)
 - `bronze_version` (1)
3. Zapisz jako `orders_bronze` w Bronze schema

**Oczekiwany rezultat:**
- Tabela Bronze z surowymi danymi JSON + audit metadata
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 1.1 - Bronze Orders

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Wczytaj surowe zamówienia JSON
orders_raw = (
 spark.read
 .format("____") # json
 .option("____", "true") # multiLine
 .load(____) # ORDERS_JSON
)

print("=== Surowe dane orders ===")
orders_raw.printSchema()
display(orders_raw.limit(3))

# Dodaj audit metadata dla Bronze
orders_bronze = (
 orders_raw
 .withColumn("____", F.____) # bronze_ingest_timestamp, current_timestamp()
 .withColumn("____", F.____) # bronze_source_file, input_file_name()
 .withColumn("____", F.lit(____)) # bronze_ingested_by, raw_user
 .withColumn("____", F.lit(____)) # bronze_version, 1
)

# Zapisz do Bronze
orders_bronze_table = f"{BRONZE_SCHEMA}.orders_bronze"

(
 orders_bronze
 .write
 .format("____") # delta
 .mode("____") # overwrite
 .option("overwriteSchema", "true")
 .saveAsTable(____) # orders_bronze_table
)

print(f"\n Bronze Orders created: {orders_bronze_table}")
print(f"Liczba rekordów: {spark.table(orders_bronze_table).count()}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 1.2: Bronze - Customers (CSV) i Products (Parquet)

**Instrukcje:**
1. Wczytaj `customers.csv` (header=true, inferSchema=true)
2. Wczytaj `products.parquet`
3. Dla obu: dodaj te same audit columns co w Zadaniu 1.1
4. Zapisz jako `customers_bronze` i `products_bronze`

**Wskazówki:**
- Użyj tego samego wzorca audit columns
- Możesz stworzyć funkcję helper dla dodania audit metadata
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 1.2 - Bronze Customers & Products

# Helper function dla audit metadata (opcjonalnie)
def add_bronze_audit(df, version=1):
 """Dodaj standardowe audit columns dla Bronze layer"""
 return (
 df
 .withColumn("bronze_ingest_timestamp", F.____)
 .withColumn("bronze_source_file", F.____)
 .withColumn("bronze_ingested_by", F.lit(____))
 .withColumn("bronze_version", F.lit(____))
 )

# Customers - CSV
customers_raw = (
 spark.read
 .format("____")
 .option("header", "____")
 .option("inferSchema", "____")
 .load(____)
)

customers_bronze = add_bronze_audit(customers_raw)
customers_bronze_table = f"{BRONZE_SCHEMA}.customers_bronze"

customers_bronze.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(customers_bronze_table)

print(f" Bronze Customers: {customers_bronze_table}")
print(f" Liczba rekordów: {spark.table(customers_bronze_table).count()}")

# Products - Parquet
products_raw = spark.read.format("____").load(____)

products_bronze = add_bronze_audit(____)
products_bronze_table = f"{BRONZE_SCHEMA}.products_bronze"

products_bronze.write.format("____").mode("____").option("overwriteSchema", "true").saveAsTable(____)

print(f"\n Bronze Products: {products_bronze_table}")
print(f" Liczba rekordów: {spark.table(products_bronze_table).count()}")

print("\n Wszystkie Bronze tables utworzone!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 2: Silver Layer - Cleaning & Validation (30 minut)

**Cel:** Transformacja Bronze → Silver z cleaning, validation, JSON flattening i deduplikacją.

### Zadanie 2.1: Silver - Orders (cleaning + validation)

**Instrukcje:**
1. Wczytaj z Bronze: `orders_bronze`
2. Cleaning & validation:
 - Deduplikacja po `order_id`
 - Filtr: `order_id` i `customer_id` NOT NULL
 - Filtr: `order_amount` > 0
 - Convert `order_date` z string na date type
 - Standaryzacja `order_status` (UPPER + TRIM)
3. Dodaj Silver metadata:
 - `silver_processed_timestamp`
 - `data_quality_flag` = "VALID"
4. Zapisz jako `orders_silver`

**Oczekiwany rezultat:**
- Oczyszczone zamówienia w Silver
- Część rekordów odfiltrowana (invalid data)
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 2.1 - Silver Orders

spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")

# Wczytaj Bronze
orders_bronze_df = spark.table(orders_bronze_table)

# Silver transformations
orders_silver = (
 orders_bronze_df
 # Deduplikacja
 .dropDuplicates([____]) # order_id
 
 # Walidacja NOT NULL
 .filter(F.col("____").isNotNull()) # order_id
 .filter(F.col("____").isNotNull()) # customer_id
 
 # Walidacja biznesowa
 .filter(F.col("____") ____ ____) # order_amount > 0
 
 # Type conversion
 .withColumn("order_date", F.____(F.col("order_date"))) # to_date
 
 # Standaryzacja
 .withColumn("order_status", F.____(F.____(F.col("order_status")))) # upper(trim())
 
 # Silver metadata
 .withColumn("____", F.current_timestamp())
 .withColumn("____", F.lit("VALID"))
)

print("=== Silver Orders - cleaned data ===")
display(orders_silver.limit(5))

# Zapisz do Silver
orders_silver_table = f"{SILVER_SCHEMA}.orders_silver"

orders_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(orders_silver_table)

bronze_count = orders_bronze_df.count()
silver_count = spark.table(orders_silver_table).count()

print(f"\n Silver Orders: {orders_silver_table}")
print(f"Bronze → Silver: {bronze_count} → {silver_count} (filtered: {bronze_count - silver_count})")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 2.2: Silver - Customers (advanced cleaning)

**Instrukcje:**
1. Wczytaj z Bronze: `customers_bronze`
2. Cleaning:
 - Deduplikacja po `customer_id`
 - Filtr: `customer_id`, `email` NOT NULL
 - Standaryzacja: `email` → lowercase, `country` → uppercase
 - Walidacja: `age` między 18 a 100
 - Convert `registration_date` na date type
3. Quality flags:
 - Dodaj kolumnę `email_domain` (extract domain z email)
 - Dodaj kolumnę `customer_segment` (based on age: 18-30="Young", 31-50="Middle", 51+="Senior")
4. Zapisz jako `customers_silver`

**Wskazówki:**
- Email domain: użyj `split` i `getItem`
- Customer segment: użyj `when().otherwise()`
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 2.2 - Silver Customers

# Wczytaj Bronze
customers_bronze_df = spark.table(customers_bronze_table)

# Silver transformations
customers_silver = (
 customers_bronze_df
 # Deduplikacja
 .dropDuplicates([____])
 
 # Walidacja NOT NULL
 .filter(F.col("____").isNotNull())
 .filter(F.col("____").isNotNull())
 
 # Standaryzacja
 .withColumn("email", F.____(F.col("email"))) # lowercase
 .withColumn("country", F.____(F.col("country"))) # upper
 
 # Walidacja wieku
 .filter((F.col("____") >= ____) & (F.col("____") <= ____)) # age between 18 and 100
 
 # Type conversion
 .withColumn("registration_date", F.____(F.col("registration_date")))
 
 # Email domain extraction
 .withColumn("email_domain", 
 F.split(F.col("____"), "@").getItem(____)) # email, 1
 
 # Customer segmentation
 .withColumn("customer_segment",
 F.when(F.col("____") <= ____, "____") # age <= 30, "Young"
 .when(F.col("____") <= ____, "____") # age <= 50, "Middle"
 .otherwise("____")) # "Senior"
 
 # Silver metadata
 .withColumn("silver_processed_timestamp", F.current_timestamp())
 .withColumn("data_quality_flag", F.lit("VALID"))
)

print("=== Silver Customers - with segmentation ===")
display(customers_silver.select("customer_id", "email", "age", "customer_segment", "email_domain").limit(5))

# Zapisz do Silver
customers_silver_table = f"{SILVER_SCHEMA}.customers_silver"

customers_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(customers_silver_table)

print(f"\n Silver Customers: {customers_silver_table}")
print(f"Liczba rekordów: {spark.table(customers_silver_table).count()}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 2.3: Silver - Products (validation + enrichment)

**Instrukcje:**
1. Wczytaj z Bronze: `products_bronze`
2. Cleaning:
 - Deduplikacja po `product_id`
 - Filtr: `product_id`, `product_name` NOT NULL
 - Walidacja: `unit_price` > 0, `stock_quantity` >= 0
 - Standaryzacja: `category` → uppercase
3. Enrichment:
 - Dodaj `stock_status`: "In Stock" (quantity > 0), "Out of Stock" (quantity = 0)
 - Dodaj `price_tier`: "Budget" (<50), "Standard" (50-200), "Premium" (>200)
4. Zapisz jako `products_silver`
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 2.3 - Silver Products

# Wczytaj Bronze
products_bronze_df = spark.table(products_bronze_table)

# Silver transformations
products_silver = (
 products_bronze_df
 # Deduplikacja
 .____([____])
 
 # Walidacja NOT NULL
 .filter(F.col("____").isNotNull())
 .filter(F.col("____").isNotNull())
 
 # Walidacja wartości
 .filter(F.col("____") > ____) # unit_price > 0
 .filter(F.col("____") >= ____) # stock_quantity >= 0
 
 # Standaryzacja
 .withColumn("category", F.____(F.col("category")))
 
 # Stock status
 .withColumn("stock_status",
 F.when(F.col("____") ____ ____, "____") # stock_quantity > 0, "In Stock"
 .otherwise("____")) # "Out of Stock"
 
 # Price tier
 .withColumn("price_tier",
 F.when(F.col("____") < ____, "____") # unit_price < 50, "Budget"
 .when(F.col("____") <= ____, "____") # unit_price <= 200, "Standard"
 .otherwise("____")) # "Premium"
 
 # Silver metadata
 .withColumn("silver_processed_timestamp", F.current_timestamp())
 .withColumn("data_quality_flag", F.lit("VALID"))
)

print("=== Silver Products - enriched ===")
display(products_silver.select("product_id", "product_name", "unit_price", "stock_quantity", "stock_status", "price_tier").limit(5))

# Zapisz do Silver
products_silver_table = f"{SILVER_SCHEMA}.products_silver"

products_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(products_silver_table)

print(f"\n Silver Products: {products_silver_table}")
print(f"Liczba rekordów: {spark.table(products_silver_table).count()}")

print("\n Wszystkie Silver tables utworzone!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 3: Gold Layer - Business Aggregates & KPIs (35 minut)

**Cel:** Utworzenie Gold layer z business-level agregacjami, KPI i raportami.

### Zadanie 3.1: Gold - Daily Sales Summary

**Instrukcje:**
1. JOIN orders_silver + customers_silver + products_silver
2. Agreguj per `order_date`:
 - `total_orders`: liczba zamówień
 - `total_revenue`: suma order_amount
 - `avg_order_value`: średnia order_amount
 - `unique_customers`: distinct customer_id
 - `unique_products`: distinct product_id
3. Dodaj `gold_created_timestamp`
4. Partycjonuj po `order_date`
5. Zapisz jako `daily_sales_summary`

**Oczekiwany rezultat:**
- Gold table z daily KPIs
- Partycjonowana dla efektywnych zapytań
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 3.1 - Gold Daily Sales Summary

spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

# Wczytaj Silver tables
orders_df = spark.table(orders_silver_table)
customers_df = spark.table(customers_silver_table)
products_df = spark.table(products_silver_table)

# Join orders + customers + products (dla kompletnego kontekstu)
# Dla daily summary nie potrzebujemy wszystkich kolumn, ale dla późniejszych analiz warto mieć JOIN
orders_enriched = (
 orders_df
 .join(customers_df, "____", "____") # customer_id, left
 .join(products_df, orders_df["____"] == products_df["____"], "left") # product_id (zakładamy że jest w orders)
)

# UWAGA: Nasze dane orders nie mają product_id - uproszczona wersja
# Agregacja Daily Sales Summary (bez products dla uproszczenia)
daily_sales = (
 orders_df
 .join(customers_df, "customer_id", "left")
 .groupBy("____") # order_date
 .agg(
 F.count("____").alias("____"), # order_id, total_orders
 F.sum("____").alias("____"), # order_amount, total_revenue
 F.avg("____").alias("____"), # order_amount, avg_order_value
 F.min("____").alias("____"), # order_amount, min_order_value
 F.max("____").alias("____"), # order_amount, max_order_value
 F.countDistinct("____").alias("____"), # customer_id, unique_customers
 F.collect_set("____").alias("____") # order_status, order_statuses (array)
 )
 .withColumn("____", F.current_timestamp())
 .orderBy("order_date")
)

print("=== Gold Daily Sales Summary ===")
display(daily_sales)

# Zapisz do Gold z partycjonowaniem
daily_sales_table = f"{GOLD_SCHEMA}.daily_sales_summary"

(
 daily_sales
 .write
 .format("delta")
 .mode("overwrite")
 .partitionBy("____") # order_date
 .option("overwriteSchema", "true")
 .saveAsTable(____)
)

print(f"\n Gold Daily Sales: {daily_sales_table}")
print(f"Liczba dni: {spark.table(daily_sales_table).count()}")

# Sprawdź partycje
print("\n=== Partycje ===")
display(spark.sql(f"SHOW PARTITIONS {daily_sales_table}"))
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 3.2: Gold - Customer Lifetime Value (CLV)

**Instrukcje:**
1. Agreguj per `customer_id`:
 - `total_orders`: liczba zamówień klienta
 - `total_spent`: suma order_amount
 - `avg_order_value`: średnia order_amount
 - `first_order_date`: min(order_date)
 - `last_order_date`: max(order_date)
 - `customer_tenure_days`: różnica dni między first i last order
2. JOIN z customers_silver dla kontekstu (country, age, segment)
3. Dodaj `clv_tier`: "High" (>1000), "Medium" (500-1000), "Low" (<500)
4. Zapisz jako `customer_lifetime_value`

**Oczekiwany rezultat:**
- Gold table z customer-level KPIs
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 3.2 - Gold Customer Lifetime Value

# Agregacja per customer
customer_orders_agg = (
 orders_df
 .groupBy("____") # customer_id
 .agg(
 F.count("____").alias("____"), # order_id, total_orders
 F.sum("____").alias("____"), # order_amount, total_spent
 F.avg("____").alias("____"), # order_amount, avg_order_value
 F.min("____").alias("____"), # order_date, first_order_date
 F.max("____").alias("____") # order_date, last_order_date
 )
 # Customer tenure
 .withColumn("customer_tenure_days",
 F.datediff(F.col("____"), F.col("____"))) # last_order_date, first_order_date
)

# JOIN z customers dla kontekstu
customer_clv = (
 customer_orders_agg
 .join(customers_df, "____", "____") # customer_id, left
 # CLV Tier
 .withColumn("clv_tier",
 F.when(F.col("____") ____ ____, "____") # total_spent > 1000, "High"
 .when(F.col("____") ____ ____, "____") # total_spent >= 500, "Medium"
 .otherwise("____")) # "Low"
 
 .withColumn("gold_created_timestamp", F.current_timestamp())
 .select(
 "customer_id", "first_name", "last_name", "email", "country", "customer_segment",
 "total_orders", "total_spent", "avg_order_value",
 "first_order_date", "last_order_date", "customer_tenure_days",
 "clv_tier", "gold_created_timestamp"
 )
 .orderBy("total_spent", ascending=False)
)

print("=== Gold Customer Lifetime Value ===")
display(customer_clv.limit(10))

# Zapisz do Gold
clv_table = f"{GOLD_SCHEMA}.customer_lifetime_value"

customer_clv.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(clv_table)

print(f"\n Gold CLV: {clv_table}")
print(f"Liczba klientów: {spark.table(clv_table).count()}")

# CLV Distribution
print("\n=== CLV Tier Distribution ===")
display(spark.table(clv_table).groupBy("clv_tier").count().orderBy("count", ascending=False))
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 3.3: Gold - Product Performance Summary

**Instrukcje:**
1. Agreguj per `category` (z products_silver):
 - `product_count`: liczba produktów w kategorii
 - `avg_price`: średnia unit_price
 - `total_stock_value`: sum(unit_price * stock_quantity)
 - `out_of_stock_count`: liczba produktów z stock_quantity = 0
2. Dodaj `category_rank` (ranking kategorii po total_stock_value)
3. Zapisz jako `product_performance_by_category`

**Oczekiwany rezultat:**
- Gold table z product/category analytics
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 3.3 - Gold Product Performance

# Agregacja per category
product_performance = (
 products_df
 .groupBy("____") # category
 .agg(
 F.count("____").alias("____"), # product_id, product_count
 F.avg("____").alias("____"), # unit_price, avg_price
 F.sum(F.col("____") * F.col("____")).alias("____"), # unit_price * stock_quantity, total_stock_value
 F.sum(F.when(F.col("____") == ____, 1).otherwise(0)).alias("____") # stock_quantity == 0, out_of_stock_count
 )
 .withColumn("gold_created_timestamp", F.current_timestamp())
)

# Dodaj category rank (window function)
window_spec = Window.orderBy(F.col("____").desc()) # total_stock_value

product_performance_ranked = (
 product_performance
 .withColumn("category_rank", F.____().over(____)) # row_number(), window_spec
 .orderBy("category_rank")
)

print("=== Gold Product Performance by Category ===")
display(product_performance_ranked)

# Zapisz do Gold
product_perf_table = f"{GOLD_SCHEMA}.product_performance_by_category"

product_performance_ranked.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(product_perf_table)

print(f"\n Gold Product Performance: {product_perf_table}")
print(f"Liczba kategorii: {spark.table(product_perf_table).count()}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 4: Optymalizacja Pipeline (20 minut)

**Cel:** Zastosowanie technik optymalizacji dla production-ready pipeline.

### Zadanie 4.1: OPTIMIZE + ZORDER na Silver tables

**Instrukcje:**
1. Uruchom `OPTIMIZE` na wszystkich Silver tables
2. Zastosuj `ZORDER BY`:
 - `orders_silver`: ZORDER BY (customer_id, order_date)
 - `customers_silver`: ZORDER BY (customer_id)
 - `products_silver`: ZORDER BY (product_id)
3. Sprawdź metryki optymalizacji

**Oczekiwany rezultat:**
- Zoptymalizowane Silver tables dla szybkich queries
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 4.1 - OPTIMIZE Silver tables

print("=== Optymalizacja Silver Tables ===\n")

# Orders Silver
print("1. Orders Silver:")
orders_optimize = spark.sql(f"""
 ____ {orders_silver_table}
 ____ BY (____, ____)
""")
display(orders_optimize)

# Customers Silver
print("\n2. Customers Silver:")
customers_optimize = spark.sql(f"""
 OPTIMIZE {____}
 ZORDER BY (____)
""")
display(customers_optimize)

# Products Silver
print("\n3. Products Silver:")
products_optimize = spark.sql(f"""
 ____ {____}
 ____ BY (____)
""")
display(products_optimize)

print("\n Wszystkie Silver tables zoptymalizowane!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 4.2: Performance comparison (przed/po optimization)

**Instrukcje:**
1. Wykonaj query na `orders_silver` z filtrem po `customer_id`
2. Sprawdź `EXPLAIN` dla zobaczenia data skipping
3. Porównaj query time (symulacja)

**Oczekiwany rezultat:**
- EXPLAIN pokazuje data skipping (pushed filters)
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 4.2 - Performance comparison

# Query z filtrem (post-ZORDER)
test_query = f"""
 SELECT order_id, customer_id, order_date, order_amount
 FROM {orders_silver_table}
 WHERE customer_id = 105
 ORDER BY order_date DESC
"""

print("=== Query Plan (po ZORDER) ===")
spark.sql(f"EXPLAIN ____ {test_query}").show(truncate=False)

# Wykonaj query
print("\n=== Query Results ===")
result = spark.sql(test_query)
display(result)

print(f"\nLiczba rekordów: {result.count()}")

# DESCRIBE DETAIL - sprawdź numFiles
print("\n=== Table Details (po OPTIMIZE) ===")
detail = spark.sql(f"DESCRIBE DETAIL {orders_silver_table}")
display(detail.select("numFiles", "sizeInBytes"))

print("\n ZORDER poprawił data skipping - mniej plików czytanych dla selective queries!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 5: Monitoring & Data Quality (15 minut)

**Cel:** Implementacja monitoringu pipeline i data quality metrics.

### Zadanie 5.1: Data Quality Metrics

**Instrukcje:**
1. Dla każdej warstwy (Bronze/Silver/Gold) oblicz:
 - Liczba rekordów
 - Liczba plików
 - Rozmiar w MB
 - Data freshness (max timestamp)
2. Utwórz summary DataFrame
3. Zapisz jako `pipeline_monitoring_summary` w Gold

**Oczekiwany rezultat:**
- Monitoring dashboard data dla pipeline
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 5.1 - Data Quality Metrics

def get_table_metrics(table_name, layer):
 """Pobierz metryki dla tabeli"""
 detail = spark.sql(f"DESCRIBE DETAIL {table_name}").collect()[0]
 
 df = spark.table(table_name)
 count = df.count()
 
 # Znajdź timestamp column (różne nazwy per layer)
 timestamp_col = None
 if "bronze_ingest_timestamp" in df.columns:
 timestamp_col = "bronze_ingest_timestamp"
 elif "silver_processed_timestamp" in df.columns:
 timestamp_col = "silver_processed_timestamp"
 elif "gold_created_timestamp" in df.columns:
 timestamp_col = "gold_created_timestamp"
 
 max_ts = df.agg(F.max(timestamp_col)).collect()[0][0] if timestamp_col else None
 
 return {
 "layer": layer,
 "table_name": table_name.split(".")[-1],
 "record_count": count,
 "num_files": detail["numFiles"],
 "size_mb": round(detail["sizeInBytes"] / (1024*1024), 2),
 "data_freshness": max_ts,
 "monitored_at": datetime.now()
 }

# Zbierz metryki dla wszystkich tabel
metrics = []

# Bronze
metrics.append(get_table_metrics(____, "____")) # orders_bronze_table, "Bronze"
metrics.append(get_table_metrics(____, "____"))
metrics.append(get_table_metrics(____, "____"))

# Silver
metrics.append(get_table_metrics(____, "____"))
metrics.append(get_table_metrics(____, "____"))
metrics.append(get_table_metrics(____, "____"))

# Gold
metrics.append(get_table_metrics(____, "____"))
metrics.append(get_table_metrics(____, "____"))
metrics.append(get_table_metrics(____, "____"))

# Utwórz summary DataFrame
monitoring_df = spark.createDataFrame(metrics)

print("=== Pipeline Monitoring Summary ===")
display(monitoring_df.orderBy("layer", "table_name"))

# Zapisz do Gold
monitoring_table = f"{GOLD_SCHEMA}.pipeline_monitoring_summary"

monitoring_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(monitoring_table)

print(f"\n Monitoring summary: {monitoring_table}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Bronze: 3 tabele (orders, customers, products) z audit metadata
- [ ] Silver: 3 tabele (orders, customers, products) - cleaned, validated, enriched
- [ ] Gold: 3 tabele (daily_sales, customer_clv, product_performance)
- [ ] Optymalizacja: OPTIMIZE + ZORDER na Silver tables
- [ ] Monitoring: pipeline_monitoring_summary w Gold
- [ ] Partycjonowanie: daily_sales_summary partitioned by order_date

### Komendy weryfikacyjne:
</VSCode.Cell>
<VSCode.Cell language="python">
# Weryfikacja końcowa pipeline

print("=" * 80)
print("WERYFIKACJA END-TO-END PIPELINE")
print("=" * 80)

# 1. Bronze Layer
print("\n1. BRONZE LAYER:")
bronze_tables = ["orders_bronze", "customers_bronze", "products_bronze"]
for table in bronze_tables:
 full_name = f"{BRONZE_SCHEMA}.{table}"
 count = spark.table(full_name).count()
 print(f" {table}: {count} records")

# 2. Silver Layer
print("\n2. SILVER LAYER:")
silver_tables = ["orders_silver", "customers_silver", "products_silver"]
for table in silver_tables:
 full_name = f"{SILVER_SCHEMA}.{table}"
 count = spark.table(full_name).count()
 print(f" {table}: {count} records")

# 3. Gold Layer
print("\n3. GOLD LAYER:")
gold_tables = ["daily_sales_summary", "customer_lifetime_value", "product_performance_by_category"]
for table in gold_tables:
 full_name = f"{GOLD_SCHEMA}.{table}"
 count = spark.table(full_name).count()
 print(f" {table}: {count} records")

# 4. Sprawdź partycjonowanie
print("\n4. PARTITIONING:")
partitions = spark.sql(f"SHOW PARTITIONS {daily_sales_table}")
print(f" daily_sales_summary partitions: {partitions.count()}")

# 5. Sprawdź optymalizację
print("\n5. OPTIMIZATION:")
for table in [orders_silver_table, customers_silver_table, products_silver_table]:
 detail = spark.sql(f"DESCRIBE DETAIL {table}").collect()[0]
 print(f" {table.split('.')[-1]}: {detail['numFiles']} files")

# 6. Data Quality Check
print("\n6. DATA QUALITY:")
print(" Silver filtering effectiveness:")
bronze_orders = spark.table(orders_bronze_table).count()
silver_orders = spark.table(orders_silver_table).count()
filtered_pct = ((bronze_orders - silver_orders) / bronze_orders * 100) if bronze_orders > 0 else 0
print(f" Orders: {filtered_pct:.1f}% filtered out (quality issues)")

# 7. Monitoring
print("\n7. MONITORING:")
if spark.catalog.tableExists(monitoring_table):
 print(f" Monitoring summary exists: {monitoring_table}")
 display(spark.table(monitoring_table))

print("\n" + "=" * 80)
print(" WSZYSTKIE TESTY PRZESZŁY POMYŚLNIE!")
print("=" * 80)
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Podsumowanie

**W tym warsztacie zbudowałeś:**

 **Kompletny pipeline Bronze → Silver → Gold:**
- **Bronze**: 3 źródła danych (JSON, CSV, Parquet) + audit trail
- **Silver**: Cleaning, validation, enrichment, segmentation
- **Gold**: Business KPIs - daily sales, customer CLV, product performance

 **Zaawansowane transformacje:**
- Type conversion i validation (dates, numerics)
- Standaryzacja (case, trim, format)
- Enrichment (segmentation, tiers, derived columns)
- Window functions (ranking)

 **Production best practices:**
- Partycjonowanie dla performance (date partitions)
- OPTIMIZE + ZORDER dla query optimization
- Monitoring i data quality metrics
- Lineage tracking (audit metadata)

**Kluczowe metryki pipeline:**

| Layer | Tables | Records | Purpose |
|-------|--------|---------|---------|
| Bronze | 3 | ~X,XXX | Raw data landing + audit |
| Silver | 3 | ~X,XXX | Cleaned, validated, enriched |
| Gold | 3 | ~XXX | Business KPIs, aggregates |

**Performance improvements:**
- OPTIMIZE: Reduced small files by XX%
- ZORDER: Improved selective queries by X-XXx
- Partitioning: Partition pruning for date queries

**Następne kroki:**
- **Delta Live Tables**: Automatyczne utrzymanie tego pipeline
- **Databricks Jobs**: Orchestration i scheduling
- **Unity Catalog**: Governance i permissions per layer
- **BI Integration**: Connect Gold tables to Power BI/Tableau

**Gratulacje! Masz produkcyjny, zoptymalizowany pipeline Medallion Architecture!** 
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Cleanup

Opcjonalnie: usuń utworzone tabele po zakończeniu warsztatu:
</VSCode.Cell>
<VSCode.Cell language="python">
# Opcjonalne czyszczenie zasobów
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# Bronze
# spark.sql(f"DROP TABLE IF EXISTS {orders_bronze_table}")
# spark.sql(f"DROP TABLE IF EXISTS {customers_bronze_table}")
# spark.sql(f"DROP TABLE IF EXISTS {products_bronze_table}")

# Silver
# spark.sql(f"DROP TABLE IF EXISTS {orders_silver_table}")
# spark.sql(f"DROP TABLE IF EXISTS {customers_silver_table}")
# spark.sql(f"DROP TABLE IF EXISTS {products_silver_table}")

# Gold
# spark.sql(f"DROP TABLE IF EXISTS {daily_sales_table}")
# spark.sql(f"DROP TABLE IF EXISTS {clv_table}")
# spark.sql(f"DROP TABLE IF EXISTS {product_perf_table}")
# spark.sql(f"DROP TABLE IF EXISTS {monitoring_table}")

# spark.catalog.clearCache()
# print("Zasoby zostały wyczyszczone")
</VSCode.Cell>
```